In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/transformers/2b-it/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b-it/2/gemma-2b-it.gguf
/kaggle/input/gemma/transformers/2b-it/2/config.json
/kaggle/input/gemma/transformers/2b-it/2/model-00001-of-00002.safetensors
/kaggle/input/gemma/transformers/2b-it/2/model-00002-of-00002.safetensors
/kaggle/input/gemma/transformers/2b-it/2/tokenizer.json
/kaggle/input/gemma/transformers/2b-it/2/tokenizer_config.json
/kaggle/input/gemma/transformers/2b-it/2/special_tokens_map.json
/kaggle/input/gemma/transformers/2b-it/2/.gitattributes
/kaggle/input/gemma/transformers/2b-it/2/tokenizer.model
/kaggle/input/gemma/transformers/2b-it/2/generation_config.json
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [3]:
training = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')[['article', 'highlights']]
training.head()

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [4]:
# !pip install -U -q transformers langchain peft bitsandbytes trl datasets==2.16.0 notebook accelerate evaluate rouge

In [5]:
from transformers import pipeline, set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate.utils import release_memory
import torch
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from datasets import Dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
import pandas as pd
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter, HTMLHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import evaluate
import transformers
from langchain.llms.base import LLM
from typing import Any
import warnings
import gc
import random
import numpy as np

warnings.filterwarnings('ignore')

# Set seed for reproducibility
set_seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Logging in HF
login(token = 'HF_TOKEN')

2024-03-29 23:12:44.932494: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 23:12:44.932672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 23:12:45.069977: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
model = "/kaggle/input/gemma/transformers/2b-it/2"

lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.padding_side = "right" # Fixing overflow issue ref: source code
model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=bnb_config)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
training = training.head(10000)
train_data = Dataset.from_pandas(training)

test_df = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")[['article', 'highlights']]
testing_data = Dataset.from_pandas(test_df.head(10))

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['article'])):
        messages = [
            {"role": "user",
             "content": "Given the following article, write a short summary of the article in 2-3 sentences:\n\nArticle: {}".format(example['article'][i])},
            {"role": "assistant",
             "content": "{}".format(example['highlights'][i])}
        ]
        output_texts.append(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False))
        
    return output_texts

# Print the first training example
print(formatting_prompts_func(train_data[:1])[0])

<start_of_turn>user
Given the following article, write a short summary of the article in 2-3 sentences:

Article: By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The di

In [14]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=82bdd79c63f113505f9231ee04b6757e4c6c2de3bfad3d288eca2a3cab2b5b3a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [15]:
import numpy as np

import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset = testing_data,
    max_seq_length=512,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=30,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        report_to='none',
        output_dir='logs',
        optim="paged_adamw_8bit"
    ),
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    peft_config=lora_config,
    formatting_func=formatting_prompts_func,
)
trainer.train()

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Step,Training Loss
1,3.309200
2,3.647000
3,3.632900
4,3.800800
5,3.599600
6,3.146300
7,2.905000
8,2.929200
9,3.114800
10,3.101800


TrainOutput(global_step=30, training_loss=2.8687292257944743, metrics={'train_runtime': 135.5696, 'train_samples_per_second': 0.885, 'train_steps_per_second': 0.221, 'total_flos': 711459208052736.0, 'train_loss': 2.8687292257944743, 'epoch': 0.01})

In [16]:
trainer.model.save_pretrained('lora_adapter')

In [17]:
trainer.evaluate()

{'eval_loss': 2.6545815467834473,
 'eval_rouge1': 0.6095,
 'eval_rouge2': 0.2388,
 'eval_rougeL': 0.4283,
 'eval_rougeLsum': 0.4513,
 'eval_gen_len': 512.0,
 'eval_runtime': 7.1917,
 'eval_samples_per_second': 1.39,
 'eval_steps_per_second': 0.278,
 'epoch': 0.01}

In [18]:
# Load the pretrained model and our LoRa adapter
base_model_name = "/kaggle/input/gemma/transformers/2b-it/2"
adapter_model_name = "/kaggle/working/lora_adapter"

model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, adapter_model_name, device_map='auto', torch_dtype=torch.float16)

# Merge the adapters into the base model so you can use the model like a normal transformers model
model = model.merge_and_unload()
model.save_pretrained('final_model')

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
model = "/kaggle/working/final_model"

# Load the HF pipeline using our newly fine-tuned Gemma 2B
pipe_finetuned = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.float16},
    device_map='auto',
    max_new_tokens=512
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
text = "Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeared as herself in an episode of The Ed Sullivan Show and three episodes of The Dinah Shore Chevy Show, her iMDB page says. She also starred in a Broadway production of The Seven Year Itch. City News Service reported that other stage credits included As You Like It, No, No, Nanette and Damn Yankees. Forrest married writer-producer Milo Frank in 1951. He died in 2004. She is survived by her niece, Sharon Durham, and nephews, Michael and Mark Feeney. Career: A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films ."
prompt = [
            {"role": "user",
             "content": "Given the following article, write a short summary of the article in 20 words:\n\nArticle: {}".format(text)},
    
        ]

prompt = pipe_finetuned.tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)


In [21]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

Sure, here is a summary of the article in 20 words:

Sally Forrest, an actress-dancer, died at 86. She was born Katherine Feeney and became a protege of Ida Lupino. She appeared in several films and television shows, including The Ed Sullivan Show and The Dinah Shore Chevy Show.


In [ ]:
import shutil
shutil.make_archive("results_1", 'zip', "/kaggle/working/final_model")
# !pip install rouge

In [19]:
from rouge import Rouge
def calculate_rouge_score(ref_summ, gen_summ):
    rouge = Rouge()

    # Calculate ROUGE scores
    scores = rouge.get_scores(gen_summ, ref_summ)

    # Extract ROUGE-1, ROUGE-2, and ROUGE-L scores
    rouge_1_score = scores[0]["rouge-1"]["f"]
    rouge_2_score = scores[0]["rouge-2"]["f"]
    rouge_l_score = scores[0]["rouge-l"]["f"]
    return rouge_1_score, rouge_2_score, rouge_l_score


In [24]:

class Node:
    def __init__(self, article, src_summ, gen_text, r1, r2, rl):
        self.article = article
        self.src_summ = src_summ
        self.gen_text = gen_text
        self.r1 = r1
        self.r2 = r2
        self.rl = rl

rouge_scores = []    
for article, summ in test_df.iterrows():
    prompt = [
                {"role": "user",
                 "content": "Given the following article, write a short summary of the article in 2-3 sentences:\n\nArticle: {}".format(article)},

            ]

    prompt = pipe_finetuned.tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    outputs = pipe_finetuned(
                prompt,
                do_sample=True,
                temperature=0.1,
                top_k=20,
                top_p=0.3,
                add_special_tokens=True
                )
    gen_sum = outputs[0]["generated_text"][len(prompt):]
    r1, r2, rl = calculate_rouge_score(str(summ), gen_sum)
    rouge_scores.append(Node(article, summ, gen_sum, r1, r2, rl))

    
rouge_scores
    


KeyboardInterrupt

